In [1]:
#import libraries.
import numpy as np
# from tensorflow.keras.models import Sequential
# from tensorflow.keras.layers import Dense
from tensorflow.python.keras.utils.data_utils import Sequence
from keras.models import Sequential
from keras.layers import Dense, LeakyReLU
from keras.wrappers.scikit_learn import KerasRegressor
import keras
from keras import regularizers, optimizers
import sklearn
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error
from keras.initializers import glorot_normal
import sounddevice as sd
import tensorflow as tf
from mir_eval import separation 
from pystoi.stoi import stoi 
import h5py
from keras.callbacks import LearningRateScheduler, Callback, ModelCheckpoint
from keras.layers.normalization import BatchNormalization
from keras.layers import Activation
import math
from sklearn.utils import shuffle
from sklearn import preprocessing
from keras.callbacks import EarlyStopping, ModelCheckpoint
import matplotlib.pyplot as plt

Using TensorFlow backend.


In [2]:
#import libraries.
# import matplotlib.pyplot as plt
# from tabulate import tabulate
import time
import os
import librosa
from librosa.core import stft, istft
import time
import pickle
from keras import backend as K
from tensorflow.keras.models import load_model
# from keras.models import load_model

In [3]:
def reconstruct(wave,angle):
    recon1 = wave*np.cos(angle)+wave*np.sin(angle)*1j
#     recon = np.sqrt(np.power(10, wave))
#     recon1 = recon*np.cos(angle)+recon*np.sin(angle)*1j
    recon = librosa.core.istft((recon1.T), hop_length=512, win_length=2048, window='hann')
    return recon

# Loading Model

In [4]:
estimator_A = load_model('Models/Two_stage/July8/saved-model-adam-046-0.0357.hdf5')

Instructions for updating:
`normal` is a deprecated alias for `truncated_normal`
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Use tf.cast instead.


# Data 

In [5]:
data_series = str(0)

In [6]:
h5f = h5py.File('TSP/Organized/concatenated/Second_set/Train2_TSP_mixed_FFT'  + data_series + '.hdf5','r')
train_input = h5f['Train2_TSP_mixed_FFT' + data_series][:]
print(train_input.shape)
h5f.close()

(1025, 474500)


In [7]:
norms = np.zeros(train_input.shape[1])
for i in range(train_input.shape[1]):
    norms[i] = np.linalg.norm(train_input[:,i])

# Predicting Training Data

In [8]:
train_input = preprocessing.normalize(train_input, norm='l2', axis=0, copy=True)

In [9]:
prediction = estimator_A.predict(train_input.T)

In [10]:
hf = h5py.File('TSP/Predicted/predicted_TSP_Mask' + data_series + '.hdf5', 'w')
hf.create_dataset('predicted_TSP_Mask' + data_series, data=prediction)
hf.close()

In [14]:
prediction.shape

(474500, 2050)

********************************
Remember to make memory free

# Predicting vaidation Data

In [11]:
h5f = h5py.File('TSP/Organized/concatenated/Valid_TSP_mixed_FFT'  + data_series + '.hdf5','r')
Valid_input = h5f['Valid_TSP_mixed_FFT' + data_series][:]
print(Valid_input.shape)
h5f.close()

(1025, 21494)


In [12]:
norms = np.zeros(Valid_input.shape[1])
for i in range(Valid_input.shape[1]):
    norms[i] = np.linalg.norm(Valid_input[:,i])

In [13]:
Valid_input = preprocessing.normalize(Valid_input, norm='l2', axis=0, copy=True)

In [14]:
prediction = estimator_A.predict(Valid_input.T)

In [15]:
hf = h5py.File('TSP/Predicted/Valid_predicted_TSP_Mask' + data_series + '.hdf5', 'w')
hf.create_dataset('Valid_predicted_TSP_Mask' + data_series, data=prediction)
hf.close()

In [16]:
prediction.shape

(21494, 2050)